<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Où sont cherchés les modules ?

## Complément - niveau basique

Pour les débutants en informatique, le plus simple est de se souvenir que si vous voulez uniquement charger vos propres modules ou packages, il suffit de les placer dans le répertoire où vous lancez la commande python. Si vous n'êtes pas sûr de cet emplacement vous pouvez le savoir en faisant :

In [ ]:
from pathlib import Path
Path.cwd()

## Complément - niveau intermédiaire

Dans ce complément nous allons voir, de manière générale, comment sont localisés (sur le disque dur) les modules que vous chargez dans python grâce à l'instruction `import` ; nous verrons aussi où placer vos propres fichiers pour qu'ils soient accessibles à python.

[Comme expliqué ici](https://docs.python.org/3/tutorial/modules.html#the-module-search-path), lorsque vous importez le module `spam`, python cherche dans cet ordre :

* un module *built-in* de nom `spam` - possiblement/probablement écrit en C,
* ou sinon un fichier `spam.py` (ou `spam/__init.py__` s'il s'agit d'un package) ; pour le localiser on utilise la variable `sys.path` (c'est-à-dire l'attribut `path` dans le module `sys`), qui est une liste de répertoires, et qui est initialisée avec, dans cet ordre :
  * le répertoire où se trouve le point d'entrée ;
  * la variable d'environnement `PYTHONPATH` ;
  * un certain nombre d'emplacements définis au moment de la compilation de python.

Ainsi sans action particulière de l'utilisateur, python trouve l'intégralité de la librairie standard, ainsi que les modules et packages installés dans le même répertoire que le fichier passé à l'interpréteur. 

La façon dont cela se présente dans l'interpréteur des notebooks peut vous induire en erreur. Aussi je vous engage à exécuter plutôt, et sur votre machine, le programme suivant : 

```python
#!/usr/bin/env python3

import sys
from pathlib import Path

def show_argv_and_path():
    print(f"le répertoire courant est {Path.cwd()}")
    print(f"le point d'entrée du programme est {sys.argv[0]}")
    print(f"la variable sys.path contient")
    for i, path in enumerate(sys.path, 1):
        print(f"{i}-ème chemin dans sys.path {path}")

show_argv_and_path()
```

En admettant que 

* vous rangez ceci le fichier `/le/repertoire/du/script/run.py`
* et que vous lancez Python depuis un répertoire différent, disons `/le/repertoire/ou/vous/etes`
* et avec une variable `PYTHONPATH` vide;

alors vous devriez observer ceci :

```bash
$ cd /le/repertoire/ou/vous/etes/
/le/repertoire/ou/vous/etes $ python3 /le/repertoire/du/script/run.py
le répertoire courant est /le/repertoire/ou/vous/etes
le point d'entrée du programme est /le/repertoire/du/script/run.py
la variable sys.path contient
1-ème chemin dans sys.path /le/repertoire/du/script
<snip>
```
*le reste dépend de votre installation*

C'est-à-dire que :

* la variable `sys.argv[0]` contient le chemin complet `/le/repertoire/du/script/run.py`,
* et le premier terme dans `sys.path` contient `/le/repertoire/du/script!`.

La [variable d'environnement](http://en.wikipedia.org/wiki/Environment_variable) PYTHONPATH est définie de façon à donner la possibilité d'étendre ces listes depuis l'extérieur, et sans recompiler l'interpréteur, ni modifier les sources. Cette possibilité s'adresse donc à l'utilisateur final - ou à son administrateur système - plutôt qu'au programmeur.

En tant que programmeur par contre, vous avez la possibilité d'étendre `sys.path` avant de faire vos `import`.

Imaginons par exemple que vous avez écrit un petit outil utilitaire qui se compose d'un point d'entrée `main.py`, et de plusieurs modules `spam.py` et `eggs.py`. Vous n'avez pas le temps de packager proprement cet outil, vous voudriez pouvoir distribuer un *tar* avec ces trois fichiers python, qui puissent s'installer n'importe où (pourvu qu'ils soient tous les trois au même endroit), et que le point d'entrée trouve ses deux modules sans que l'utilisateur ait à s'en soucier.

Imaginons donc ces trois fichiers installés sur machine de l'utilisateur dans :

```bash
/usr/share/utilitaire/
                      main.py
                      spam.py
                      eggs.py
```

Si vous ne faites rien de particulier, c'est-à-dire que `main.py` contient juste

```python
import spam, eggs
```

Alors le programme ne fonctionnera **que s'il est lancé depuis `/usr/share/utilitaire`**, ce qui n'est pas du tout pratique. 

Pour contourner cela on peut écrire dans `main.py` quelque chose comme :

```python
# on récupère le répertoire où est installé le point d'entrée
from pathlib import Path

directory_installation = Path(__file__).parent
    
# et on l'ajoute au chemin de recherche des modules
import sys
sys.path.append(directory_installation)
    
# maintenant on peut importer spam et eggs de n'importe où
import spam, eggs
```

### Distribuer sa propre librairie avec `setuptools`

Notez bien que l'exemple précédent est **uniquement donné à titre d'illustration** pour décortiquer la mécanique d'utilisation de `sys.path`. 

Ce n'est pas une technique recommandée dans le cas général. On préfère en effet de beaucoup diffuser une application python, ou une librairie, sous forme de packaging en utilisant le [module setuptools](https://pypi.python.org/pypi/setuptools). Il s'agit d'un outil qui **ne fait pas partie de la librairie standard**, et qui supplante `distutils` qui lui, fait partie de la distribution standard mais qui est tombé en déshérence au fil du temps.

`setuptools` permet au programmeur d'écrire - dans un fichier qu'on appelle traditionnellement `setup.py` - le contenu de son application ; grâce à quoi on peut ensuite de manière unifiée :

* installer l'application sur une machine à partir des sources ;
* préparer un package de l'application ;
* diffuser le package dans [l'infrastructure PyPI](https://pypi.python.org/pypi) ;
* installer le package depuis PyPI en utilisant [`pip3`](http://pip.readthedocs.org/en/latest/installing.html).

Pour installer `setuptools`, comme d'habitude vous pouvez faire simplement :

```bash
pip3 install setuptools
```